In [2]:
# ==================================================
# ✅ Real-Time Performance Benchmark: Ensemble (MESNet) Only
# ==================================================
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import time
import os
import pandas as pd

# --------------------------
# Device setup
# --------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# --------------------------
# Transformations and Dataloaders
# --------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# --------------------------
# Define Base Models
# --------------------------
def get_efficientnet():
    model = torchvision.models.efficientnet_v2_s(weights="IMAGENET1K_V1")
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
    return model

def get_shufflenet():
    model = torchvision.models.shufflenet_v2_x1_0(weights="IMAGENET1K_V1")
    model.fc = nn.Linear(model.fc.in_features, 10)
    return model

def get_mobilenet():
    model = torchvision.models.mobilenet_v3_small(weights="IMAGENET1K_V1")
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)
    return model

# --------------------------
# Define Ensemble (MESNet)
# --------------------------
class EnsembleNet(nn.Module):
    def __init__(self, effnet, shufflenet, mobilenet):
        super(EnsembleNet, self).__init__()
        self.effnet = effnet
        self.shufflenet = shufflenet
        self.mobilenet = mobilenet

    def forward(self, x):
        with torch.no_grad():
            x1 = self.effnet(x)
            x2 = self.shufflenet(x)
            x3 = self.mobilenet(x)
        return (x1 + x2 + x3) / 3  # Average fusion (MESNet)

# --------------------------
# Performance Measurement Function
# --------------------------
def measure_performance(model, dataloader, num_samples=100):
    model.to(device)
    model.eval()
    total_time = 0
    total_images = 0

    # Reset GPU memory tracking
    if device == 'cuda':
        torch.cuda.reset_peak_memory_stats(device)

    with torch.no_grad():
        for i, (images, _) in enumerate(dataloader):
            if i * images.size(0) >= num_samples:
                break
            images = images.to(device)
            start_time = time.time()
            outputs = model(images)
            if device == 'cuda':
                torch.cuda.synchronize()
            total_time += (time.time() - start_time)
            total_images += images.size(0)

    # Compute metrics
    avg_latency = (total_time / total_images) * 1000   # ms/image
    throughput = total_images / total_time             # images/sec

    # Accurate memory usage (GPU > CPU)
    if device == 'cuda':
        mem_usage = torch.cuda.max_memory_allocated(device) / (1024 ** 2)  # MB
    else:
        import psutil
        process = psutil.Process(os.getpid())
        mem_usage = process.memory_info().rss / (1024 ** 2)

    return avg_latency, throughput, mem_usage

# --------------------------
# Run Only Ensemble Benchmark
# --------------------------
print("\n🔍 Measuring performance for Ensemble (MESNet) ...")
effnet = get_efficientnet()
shufflenet = get_shufflenet()
mobilenet = get_mobilenet()
ensemble_model = EnsembleNet(effnet, shufflenet, mobilenet)

latency, throughput, memory = measure_performance(ensemble_model, testloader, num_samples=100)

# --------------------------
# Display Results
# --------------------------
df = pd.DataFrame([["Ensemble (MESNet)", latency, throughput, memory]],
                  columns=["Model", "Latency (ms/img)", "Throughput (img/sec)", "Memory (MB)"])

print("\n==============================")
print("📊 Ensemble (MESNet) Real-Time Performance")
print("==============================")
print(df.to_string(index=False))
print("==============================")

# Real-time check
if latency <= 33.3:
    print(f"✅ Real-Time Capable (Latency {latency:.2f} ms < 33.3 ms threshold)")
else:
    print(f"❌ Not Real-Time (Latency {latency:.2f} ms > 33.3 ms threshold)")
print("==============================")

# Save results
df.to_csv("mesnet_realtime_only.csv", index=False)
print("📁 Results saved to 'mesnet_realtime_only.csv'")



Using device: cuda

🔍 Measuring performance for Ensemble (MESNet) ...

📊 Ensemble (MESNet) Real-Time Performance
            Model  Latency (ms/img)  Throughput (img/sec)  Memory (MB)
Ensemble (MESNet)          2.254844            443.489717   462.821777
✅ Real-Time Capable (Latency 2.25 ms < 33.3 ms threshold)
📁 Results saved to 'mesnet_realtime_only.csv'
